# Classification

In [ ]:
import pandas as pd
import sklearn
import sklearn.model_selection
import sklearn.datasets
import vowpalwabbit

In [ ]:
iris_dataset = sklearn.datasets.load_iris()
iris_dataframe = pd.DataFrame(
    data=iris_dataset.data, columns=iris_dataset.feature_names
)
# vw expects labels starting from 1
iris_dataframe["y"] = iris_dataset.target + 1
training_data, testing_data = sklearn.model_selection.train_test_split(
    iris_dataframe, test_size=0.2
)

In [ ]:
def to_vw_format(row):
    res = f"{int(row.y)} |"
    for idx, value in row.drop(["y"]).items():
        feature_name = idx.replace(" ", "_").replace("(", "").replace(")", "")
        res += f" {feature_name}:{value}"
    return res

## Vowpal Wabbit input format

Vowpal Wabbit has its own input format we can use. Let's see what it looks like.

In [ ]:
for ex in training_data.head(10).apply(to_vw_format, axis=1):
    print(ex)

In [ ]:
vw = vowpalwabbit.Workspace("--oaa 3 --quiet")

# learn from training set with multiple passes
for example in training_data.apply(to_vw_format, axis=1):
    vw.learn(example)

# predict from the testing set
predictions = []
for example in testing_data.apply(to_vw_format, axis=1):
    predicted_class = vw.predict(example)
    predictions.append(predicted_class)

In [ ]:
accuracy = len(testing_data[testing_data.y == predictions]) / len(testing_data)

print(f"Model accuracy {accuracy}")